In [2]:
!pip install torchvision

  Using cached torchvision-0.22.1-cp311-cp311-win_amd64.whl.metadata (6.1 kB)
Using cached torchvision-0.22.1-cp311-cp311-win_amd64.whl (1.7 MB)



[notice] A new release of pip is available: 24.0 -> 25.1.1
[notice] To update, run: C:\Program Files\Python311\python.exe -m pip install --upgrade pip


In [3]:
import os
import numpy as np
import random
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import pandas as pd
from PIL import Image
from tqdm import tqdm
from sklearn.model_selection import train_test_split
import shutil
import os
print(os.getcwd())
import torch.nn as nn

Z:\קבוצה 3\python_AI\Gender


In [4]:
from PIL import Image
import torch
from torchvision import transforms, models

# השתמשי ב-transform של התמונות ב-validation/test
val_test_transforms = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

def predict_image(image_path):
    # טען את המודל והמשקלים
    model = models.resnet18(pretrained=False)
    model.fc = nn.Sequential(
        nn.Linear(model.fc.in_features, 128),
        nn.ReLU(),
        nn.Dropout(0.5),
        nn.Linear(128, 1)
    )
    model.load_state_dict(torch.load("my_model.pt", map_location=torch.device('cpu')))
    model.eval()

    # טען ועבד את התמונה
    image = Image.open(image_path).convert("RGB")
        # שינוי גודל תוך שמירה על יחס גובה-רוחב
    image.thumbnail((224, 224), Image.LANCZOS)
    
    # יצירת תמונה חדשה עם רקע לבן
    background = Image.new('RGB', (224, 224), (255, 255, 255))
    img_w, img_h = image.size
    offset = ((224 - img_w) // 2, (224 - img_h) // 2)
    background.paste(image, offset)
    image = val_test_transforms(background)
    image = image.unsqueeze(0)  # הוספת מימד batch

    # חיזוי
    with torch.no_grad():
        output = model(image)
        prob = torch.sigmoid(output).item()
        pred = 1 if prob > 0.5 else 0
        label = "male" if pred == 1 else "female"
        print(f"Prediction: {label} (probability: {prob:.4f})")
        return label



In [5]:
predict_image(rf"C:\\Users\\215556754\\Desktop\\non-child-1.png")

C:\Program Files\Python311\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
C:\Program Files\Python311\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


FileNotFoundError: [Errno 2] No such file or directory: 'C:\\Users\\215556754\\Desktop\\non-child-1.png'